# Algorithm Testing
A notebook used to compare algorithm performance.


In [1]:
### Imports
from nitools import utils 
from nitools.classifiers import PAE_RVFL, ELM, AE_ELM 
from nitools.models import LeNet5, ResNet18
from nitools.operations import resetseed

import numpy as np
import random
import torch
import time as t

## 1. MNIST

Dataset of greyscale 28x28 images of handwritten digits. 

*Train: 60,000*, *Test: 10,000*, *10 Classes*, *784 Inputs*


In [2]:
# Import dataset
mnist = utils.load_mnist()

mnist_in = 784
mnist_class = 10

### pAE-RVFL

In [4]:
device = torch.device("cpu")

SEED = 22
resetseed(SEED)

pae_rvfl = PAE_RVFL.PAE_RVFL(
    784, 
    400, 
    mnist_class, 
    subnets=1,
    device=device, 
    r=(1,1), sc=0.5, sb=0.5, c=1000, ae_iters=3)

lenet = LeNet5.LeNet5(pae_rvfl, weight_train=True)

X = torch.from_numpy(mnist['train_X']).float()[:60000]
y = torch.from_numpy(mnist['train_y']).float()[:60000]

train_start = t.time()
result = lenet.train(X, y)
train_end = t.time()
tX = torch.from_numpy(mnist['test_X']).float()
pred = lenet.predict(tX)[:10000]
pred_arg = np.zeros(10000)

for i in range(len(pred)):
    pred_arg[i] = np.argmax(pred[i])

print(f"train: {train_end-train_start}")
utils.evaluation_summary('lenet PAE-RVFL - MNIST', pred_arg, mnist['test_y'])

train: 24.796189785003662
Evaluation for: lenet PAE-RVFL - MNIST
Classifier 'lenet PAE-RVFL - MNIST' has Acc=0.945 P=0.944 R=0.945 F1=0.944
              precision    recall  f1-score   support

         0.0      0.990     0.950     0.970      1021
         1.0      0.989     0.923     0.954      1216
         2.0      0.913     0.957     0.935       984
         3.0      0.956     0.942     0.949      1025
         4.0      0.922     0.963     0.942       940
         5.0      0.942     0.950     0.946       884
         6.0      0.961     0.948     0.954       972
         7.0      0.899     0.960     0.929       962
         8.0      0.936     0.941     0.939       969
         9.0      0.935     0.918     0.926      1027

    accuracy                          0.945     10000
   macro avg      0.944     0.945     0.944     10000
weighted avg      0.946     0.945     0.945     10000


Confusion matrix:
 [[ 970    1    0    0    0    1    4    1    3    0]
 [   0 1122    2    2    1  

### ELM

In [ ]:
SEED = 22
resetseed(SEED)

# Get ELM results
mnist_elm = AE_ELM.AE_ELM(  # TODO: batchnorm input weights to ae?
    input_size=mnist_in,
    h_size=1000,
    output_size=mnist_class,
    ae_iters=5)

lenet = LeNet5.LeNet5(mnist_elm, weight_train=True)

X = torch.from_numpy(mnist['train_X']).float()[:60000]
y = torch.from_numpy(mnist['train_y']).float()[:60000]

train_start = t.time()
result = lenet.train(X, y)
train_end = t.time()
tX = torch.from_numpy(mnist['test_X']).float()
pred = lenet.predict(tX)[:10000]
pred_arg = np.zeros(10000)

for i in range(len(pred)):
    pred_arg[i] = np.argmax(pred[i])


print(f"train: {train_end-train_start}")
utils.evaluation_summary('lenet AE-ELM - MNIST', pred_arg, mnist['test_y'])

In [ ]:
SEED = 22
resetseed(SEED)

# Get ELM results
mnist_elm = ELM.ELM(  # TODO: batchnorm input weights to ae?
    input_size=mnist_in,
    h_size=1000,
    output_size=mnist_class)

lenet = LeNet5.LeNet5(mnist_elm, weight_train=True)

X = torch.from_numpy(mnist['train_X']).float()[:60000]
y = torch.from_numpy(mnist['train_y']).float()[:60000]

train_start = t.time()
result = lenet.train(X, y)
train_end = t.time()
tX = torch.from_numpy(mnist['test_X']).float()
pred = lenet.predict(tX)[:10000]
pred_arg = np.zeros(10000)

for i in range(len(pred)):
    pred_arg[i] = np.argmax(pred[i])


print(f"train: {train_end-train_start}")
utils.evaluation_summary('lenet  - MNIST', pred_arg, mnist['test_y'])

## 2. CIFAR-10

Dataset of colour (3-dimension) 32x32 images of objects of 10 classes:

0.	airplane
1.	automobile
2.	bird
3.	cat
4.	deer
5.	dog
6.	frog
7.	horse
8.	ship
9.	truck

*Train: 60,000*, *Test: 10,000*, *10 Classes*, *784 Inputs*

In [ ]:
# Import dataset
cifar10 = utils.load_cifar10()

cifar_in = 32*32*3
cifar_class = 10

### SPAE-RVFL

In [ ]:
device = torch.device("cpu")

pae_rvfl = SPAE_RVFL.SPAE_RVFL(
    cifar_in, 
    cifar_class, 
    [100], 
    subnets=[5], 
    device=device, 
    r=[(1,1), (1,1)], sc=0.5, sb=0.5)

lenet = LeNet5.LeNet5_3D(pae_rvfl)

X = torch.from_numpy(cifar10['train_X']).float()[:5000]
y = torch.from_numpy(cifar10['train_y']).float()[:5000]

result = lenet.train(X, y)
tX = torch.from_numpy(cifar10['test_X']).float()
pred = lenet.predict(tX)[:10000]
pred_arg = np.zeros(10000)

for i in range(len(pred)):
    pred_arg[i] = np.argmax(pred[i])

utils.evaluation_summary('lenet PAE-RVFL - MNIST', pred_arg, cifar10['test_y'])

In [ ]:
# Show results